# im2latex(S): Tokenizer

&copy; Copyright 2017 Sumeet S Singh

    This file is part of im2latex solution by Sumeet S Singh.

    This program is free software: you can redistribute it and/or modify
    it under the terms of the Affero GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    Affero GNU General Public License for more details.

    You should have received a copy of the Affero GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string
from PIL import Image

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 100
pd.options.display.width = 160
data_dir = '../data/generated2'
image_folder = os.path.join(data_dir,'formula_images')

#### Characterset Chart for Reference
##### ASCII Control Characters

                        CTRL   (^D means to hold the CTRL key and hit d)
    Oct  Dec Char  Hex  Key     Comments
    \000   0  NUL  \x00  ^@ \0 (Null byte)
    \001   1  SOH  \x01  ^A    (Start of heading)
    \002   2  STX  \x02  ^B    (Start of text)
    \003   3  ETX  \x03  ^C    (End of text) (see: UNIX keyboard CTRL)
    \004   4  EOT  \x04  ^D    (End of transmission) (see: UNIX keyboard CTRL)
    \005   5  ENQ  \x05  ^E    (Enquiry)
    \006   6  ACK  \x06  ^F    (Acknowledge)
    \007   7  BEL  \x07  ^G    (Ring terminal bell)
    \010   8   BS  \x08  ^H \b (Backspace)  (\b matches backspace inside [] only)
                                            (see: UNIX keyboard CTRL)
    \011   9   HT  \x09  ^I \t (Horizontal tab)
    \012  10   LF  \x0A  ^J \n (Line feed)  (Default UNIX NL) (see End of Line below)
    \013  11   VT  \x0B  ^K    (Vertical tab)
    \014  12   FF  \x0C  ^L \f (Form feed)
    \015  13   CR  \x0D  ^M \r (Carriage return)  (see: End of Line below)
    \016  14   SO  \x0E  ^N    (Shift out)
    \017  15   SI  \x0F  ^O    (Shift in)
    \020  16  DLE  \x10  ^P    (Data link escape)
    \021  17  DC1  \x11  ^Q    (Device control 1) (XON) (Default UNIX START char.)
    \022  18  DC2  \x12  ^R    (Device control 2)
    \023  19  DC3  \x13  ^S    (Device control 3) (XOFF)  (Default UNIX STOP char.)
    \024  20  DC4  \x14  ^T    (Device control 4)
    \025  21  NAK  \x15  ^U    (Negative acknowledge)  (see: UNIX keyboard CTRL)
    \026  22  SYN  \x16  ^V    (Synchronous idle)
    \027  23  ETB  \x17  ^W    (End of transmission block)
    \030  24  CAN  \x18  ^X    (Cancel)
    \031  25  EM   \x19  ^Y    (End of medium)
    \032  26  SUB  \x1A  ^Z    (Substitute character)
    \033  27  ESC  \x1B  ^[    (Escape)
    \034  28  FS   \x1C  ^\    (File separator, Information separator four)
    \035  29  GS   \x1D  ^]    (Group separator, Information separator three)
    \036  30  RS   \x1E  ^^    (Record separator, Information separator two)
    \037  31  US   \x1F  ^_    (Unit separator, Information separator one)
    \177 127  DEL  \x7F  ^?    (Delete)  (see: UNIX keyboard CTRL)
    
    string.printable = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
    string.whitespace = '\t\n\x0b\x0c\r '

In [3]:
def loadImageList(filepath):
    df=pd.read_table(filepath, header=None, 
                     names=['id', 'name', 'type'], 
                     delim_whitespace=True, 
                     usecols=('id','name'), 
                     dtype={'id':int, 'name':str, 'type':str})
    return df

def getImageDetails(data_dir):
    try:
        image_details = pd.read_csv(os.path.join(data_dir,'image_details.csv'),
                                   index_col=0)
        return image_details
    except:
        pass
    widths=[]
    heights=[]
    filenames=[]
    imageList = loadImageList(os.path.join(data_dir,'im2latex.lst'))
    for i in range(imageList.shape[0]):
        try:
            image_name = imageList.iloc[i,1] + '.png'
            im = Image.open(os.path.join(image_folder,image_name))
            widths.append(im.size[0])
            heights.append(im.size[1])
            filenames.append(image_name)
        except Exception as e:
            #print(e)
            pass
    print(len(widths), len(filenames), len(heights))
    dff = pd.DataFrame({'filename':filenames, 'width':widths, 'height':heights})
    dff.to_csv(os.path.join(data_dir,'image_details.csv'))
    return dff


def getDatasetDetails(data_dir):
    try:
        image_details = pd.read_pickle(os.path.join(data_dir,'df_image_details.pkl'))
        return image_details
    except:
        pass
    widths=[]
    heights=[]
    formula_lens=[]
    datasetDF = pd.read_pickle(os.path.join(data_dir,'im2latex_map.pkl'))
    for _, row in datasetDF.iterrows():
        image_name = row.image
        im = Image.open(os.path.join(image_folder,image_name))
        widths.append(im.size[0])
        heights.append(im.size[1])
        formula_lens.append(len(row.latex))
    print(len(widths), len(heights))
    datasetDF = datasetDF.assign(width=widths, height=heights, formula_len=formula_lens)
    datasetDF.to_pickle(os.path.join(data_dir,'df_image_details.pkl'))
    return datasetDF

df_image_details = getDatasetDetails(data_dir)

In [4]:
def get_df_clean(data_dir_, df_image_details_):
    NONPRINTABLE_CHARS_RE = r'[^\\' + string.printable + r']'
    DELETE_RE = re.compile(r".\x7F")
    PERCENTS_RE = r'%'
    try:
        return pd.read_pickle(os.path.join(data_dir_,'df_clean.pkl'))
    except Exception as e:
        print e    
        df = df_image_details_
        # Make sure everything's ascii
        # Coalesce whitespace to a single space
        # Strip whitespace from the sides
        # Strip percent signs from the sides
        # Discard strings with non-printable characters
        # Discard strings with embedded percent signs (because textogif ignores everything after the % sign)
        cleaned = df.latex.str.decode('ascii').str.encode('ascii').str.replace(r"\s+", ' ').str.strip().str.strip('%')
        df = df.assign(latex_ascii=cleaned, latex_ascii_len=cleaned.str.len())
        bad1 = df.latex.str.contains(NONPRINTABLE_CHARS_RE)
        print 'nonprintables #: ', bad1.shape
        bad2 = df.latex.str.contains(PERCENTS_RE)
        print 'percents #: ', bad1.shape
        good = ~(bad1 | bad2)
        print 'good #: ', good.shape
        df = df[good]
        df.to_pickle((os.path.join(data_dir_,'df_clean.pkl')))
        return df

In [5]:
df_clean = get_df_clean(data_dir, df_image_details)
display(df_clean.shape)

(99600, 8)

In [6]:
df_clean

,image,formula_name,latex,formula_len,height,width,latex_ascii,latex_ascii_len
0,450f7c1496143fd_basic.png,450f7c1496143fd_basic,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta}\t\int_{-\epsilon}^\infty dl' {\rm e}^{-l'\zeta}\t...,178,78,738,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta} \int_{-\epsilon}^\infty dl' {\rm e}^{-l'\zeta} ll...,177
1,868d5037af9e4b4_basic.png,868d5037af9e4b4_basic,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 + \alpha^{2}}\lbrace dr^2+r^2d\theta^2+r^2sin^2\the...,199,94,962,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 + \alpha^{2}}\lbrace dr^2+r^2d\theta^2+r^2sin^2\the...,198
2,af0b6c3ee18804a_basic.png,af0b6c3ee18804a_basic,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\widetilde{G}_n{(-a)^n\over2^{2n-1}}\label{H4},89,87,291,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\widetilde{G}_n{(-a)^n\over2^{2n-1}}\label{H4},89
3,dda45eca6d32fa3_basic.png,dda45eca6d32fa3_basic,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H)_{ijk} = 0 ,",59,35,405,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H)_{ijk} = 0 ,",59
4,67eb249ed1c20d2_basic.png,67eb249ed1c20d2_basic,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^{(3)}} \left(\sqrt{n} +\sqrt{\bar{n}}\right)\label...,100,60,521,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^{(3)}} \left(\sqrt{n} +\sqrt{\bar{n}}\right)\label...,100
5,89ef1bacdfcca24_basic.png,89ef1bacdfcca24_basic,"\hat N_3 = \sum\sp f_{j=1}a_j\sp {\dagger} a_j \,. \label{c5}",61,102,206,"\hat N_3 = \sum\sp f_{j=1}a_j\sp {\dagger} a_j \,. \label{c5}",61
6,fd81bdf1ebcf996_basic.png,fd81bdf1ebcf996_basic,+ \int\!\!d^D\!z_1 d^D\!z_2 d^D\!z_3 \left. \frac{\delta^2 W}{\delta j(x) \delta j(z_1)} \...,323,81,1077,"+ \int\!\!d^D\!z_1 d^D\!z_2 d^D\!z_3 \left. \frac{\delta^2 W}{\delta j(x) \delta j(z_1)} \, \fra...",273
7,6135540f1af5ff3_basic.png,6135540f1af5ff3_basic,"\,^{*}d\,^{*}H=\kappa \,^{*}d\phi = J_B . \label{bfm19}",56,30,282,"\,^{*}d\,^{*}H=\kappa \,^{*}d\phi = J_B . \label{bfm19}",55
8,92656e604bed774_basic.png,92656e604bed774_basic,{\phi''\over A} +{1\over A}\left( -{1\over 2}{A'\over A}+2{B'\over B}+{2\over r}\right)\phi'-{2 ...,159,80,817,{\phi''\over A} +{1\over A}\left( -{1\over 2}{A'\over A}+2{B'\over B}+{2\over r}\right)\phi'-{2 ...,159
9,892756cc4445c69_basic.png,892756cc4445c69_basic,\label{maxw}\partial_{\mu} (F^{\mu\nu}-ej^{\mu}x^{\nu})=0 .,59,35,296,\label{maxw}\partial_{\mu} (F^{\mu\nu}-ej^{\mu}x^{\nu})=0 .,59


In [7]:
class TokenDict(object):
    def __init__(self):
        self._tokens = {}
    
    def account(self, token_list):
        for token in token_list:
            self._count(token)
            
    def _count(self, token):
        if token in self._tokens:
            self._tokens[token] += 1
        else:
            self._tokens[token] = 1
        return 1
    
    @property
    def dict(self):
        return self._tokens
    
    @property
    def tokens(self):
        return sorted(self._tokens.keys())
            
def get_vocabulary(df_, data_dir_):
    try:
        df_vocab = pd.read_pickle(os.path.join(data_dir_,'df_vocab.pkl'))
        df_tokenized = pd.read_pickle(os.path.join(data_dir_,'df_tokenized.pkl'))
    except Exception as e:
        print e
        ## Split latex into tokens. Isolate latex commands first - i.e.
        ## (optionally even number of backslashes) followed by one backslash followed by letters.
        ## Everything else is a one-character token in itself.
        LATEX_RE = re.compile(r"(?:(?<=\\\\\\\\\\\\)\\[a-zA-Z]+)|(?:(?<=\\\\\\\\)\\[a-zA-Z]+)|(?:(?<=\\\\)\\[a-zA-Z]+)|(?:(?<!\\)\\[a-zA-Z]+)|.")
        sr_token = df_.latex_ascii.str.findall(LATEX_RE)
        df_tokenized = df_.assign(latex_tokenized=sr_token)
        ## Aggregate the tokens
        vocab = TokenDict()
        sr_token.agg(lambda l: vocab.account(l))
        ## Sort and save
        tokens = []; 
        count = []
        for t in vocab.tokens:
            tokens.append(t)
            count.append(vocab.dict[t])
        ## Assign token-ids. Start with 2. Reserve 0 as a 'null' token, 1 as begin-sequence token
        df_vocab = pd.DataFrame({'id':range(2,len(tokens)+2), 'freq':count}, index=tokens, columns=['id', 'freq'])
        ## Now ensure that space is the last ID. This is required by the CTC decoder.
        max_id = df_vocab.id.max()
        #print 'max_id =', max_id, type(max_id)
        max_idx = df_vocab[df_vocab.id == max_id].index[0]
        #print 'max_idx=', max_idx, type(max_idx)
        space_id = df_vocab.loc[' '].id
        #print 'space_id=', space_id, type(space_id)        ## Persist to disk
        df_vocab.loc[' '].id = max_id
        df_vocab.loc[max_idx].id = space_id
        print 'swapped ids %d and %d'%(max_id, space_id)
        display(df_vocab.loc[' '])
        display(df_vocab.loc[max_idx])
        df_vocab.to_pickle(os.path.join(data_dir_,'df_vocab.pkl'))
        df_tokenized.to_pickle(os.path.join(data_dir_,'df_tokenized.pkl'))
        
    return df_vocab, df_tokenized

In [8]:
df_vocab, df_tokenized = get_vocabulary(df_clean, data_dir)

In [9]:
df_vocab

,id,freq
,556,863152
!,3,8052
"""",4,62
#,5,93
$,6,3908
&,7,29294
',8,20412
(,9,204696
),10,204723
*,11,6578


In [10]:
#df_vocab[df_vocab.index.str.contains(r'\\')]

In [11]:
print df_clean.latex_ascii[df_clean.latex.str.contains(r'\\\\\\\\\\\\\\\\\\\\')].count()

0


In [12]:
def get_word2id(df_tokenized_, df_vocab_, data_dir_):
    try:
        return pd.read_pickle(os.path.join(data_dir, 'df_word2id.pkl'))
    except Exception as e:
        print e        
        word2id = df_vocab_.id.to_dict()
        sr_word2id = df_tokenized_.latex_tokenized.apply(lambda l: map(lambda t: word2id[t], l))
        df_ = df_tokenized_.assign(word2id=sr_word2id, word2id_len=sr_word2id.str.len())
        df_.to_pickle(os.path.join(data_dir_, 'df_word2id.pkl'))
        return df_

In [13]:
df_word2id = get_word2id(df_tokenized, df_vocab, data_dir)

In [14]:
df_word2id.shape

(99600, 11)

In [15]:
df_word2id

,image,formula_name,latex,formula_len,height,width,latex_ascii,latex_ascii_len,latex_tokenized,word2id,word2id_len
0,450f7c1496143fd_basic.png,450f7c1496143fd_basic,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta}\t\int_{-\epsilon}^\infty dl' {\rm e}^{-l'\zeta}\t...,178,78,738,\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta} \int_{-\epsilon}^\infty dl' {\rm e}^{-l'\zeta} ll...,177,"[\int, _, {, -, \epsilon, }, ^, \infty, , d, l, \, :, , {, \rm, , e, }, ^, {, -, l, \zeta, },...","[258, 525, 553, 14, 213, 555, 524, 257, 556, 530, 538, 61, 27, 556, 553, 411, 556, 531, 555, 524...",112
1,868d5037af9e4b4_basic.png,868d5037af9e4b4_basic,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 + \alpha^{2}}\lbrace dr^2+r^2d\theta^2+r^2sin^2\the...,199,94,962,ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 + \alpha^{2}}\lbrace dr^2+r^2d\theta^2+r^2sin^2\the...,198,"[d, s, ^, {, 2, }, , =, , (, 1, , -, , {, q, c, o, s, \theta, \over, , r, }, ), ^, {, 2, \o...","[530, 545, 524, 553, 19, 555, 556, 30, 556, 9, 18, 556, 14, 556, 553, 543, 529, 541, 545, 475, 3...",125
2,af0b6c3ee18804a_basic.png,af0b6c3ee18804a_basic,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\widetilde{G}_n{(-a)^n\over2^{2n-1}}\label{H4},89,87,291,\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\widetilde{G}_n{(-a)^n\over2^{2n-1}}\label{H4},89,"[\widetilde, \gamma, _, {, \rm, , h, o, p, f, }, \simeq, \sum, _, {, n, >, 0, }, \widetilde, {,...","[518, 232, 525, 553, 411, 556, 534, 541, 542, 532, 555, 431, 455, 525, 553, 540, 31, 17, 555, 51...",47
3,dda45eca6d32fa3_basic.png,dda45eca6d32fa3_basic,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H)_{ijk} = 0 ,",59,35,405,"({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H)_{ijk} = 0 ,",59,"[(, {, \cal, , L, }, _, a, , g, ), _, {, i, j, }, , =, , 0, ,, , \, , \, , \, , \, , (,...","[9, 553, 160, 556, 45, 555, 525, 527, 556, 533, 10, 525, 553, 535, 536, 555, 556, 30, 556, 17, 1...",53
4,67eb249ed1c20d2_basic.png,67eb249ed1c20d2_basic,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^{(3)}} \left(\sqrt{n} +\sqrt{\bar{n}}\right)\label...,100,60,521,S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^{(3)}} \left(\sqrt{n} +\sqrt{\bar{n}}\right)\label...,100,"[S, _, {, s, t, a, t, }, , =, , 2, \pi, , \sqrt, {, N, _, 5, ^, {, (, 1, ), }, , N, _, 5, ^,...","[52, 525, 553, 545, 546, 527, 546, 555, 556, 30, 556, 19, 378, 556, 447, 553, 47, 525, 22, 524, ...",69
5,89ef1bacdfcca24_basic.png,89ef1bacdfcca24_basic,"\hat N_3 = \sum\sp f_{j=1}a_j\sp {\dagger} a_j \,. \label{c5}",61,102,206,"\hat N_3 = \sum\sp f_{j=1}a_j\sp {\dagger} a_j \,. \label{c5}",61,"[\hat, , N, _, 3, , =, , \sum, \sp, , f, _, {, j, =, 1, }, a, _, j, \sp, , {, \dagger, }, ...","[239, 556, 47, 525, 20, 556, 30, 556, 455, 442, 556, 532, 525, 553, 536, 30, 18, 555, 527, 525, ...",40
6,fd81bdf1ebcf996_basic.png,fd81bdf1ebcf996_basic,+ \int\!\!d^D\!z_1 d^D\!z_2 d^D\!z_3 \left. \frac{\delta^2 W}{\delta j(x) \delta j(z_1)} \...,323,81,1077,"+ \int\!\!d^D\!z_1 d^D\!z_2 d^D\!z_3 \left. \frac{\delta^2 W}{\delta j(x) \delta j(z_1)} \, \fra...",273,"[+, , \int, \, !, \, !, d, ^, D, \, !, z, _, 1, , d, ^, D, \, !, z, _, 2, , d, ^, D, \, !, z,...","[12, 556, 258, 61, 3, 61, 3, 530, 524, 37, 61, 3, 552, 525, 18, 556, 530, 524, 37, 61, 3, 552, 5...",166
7,6135540f1af5ff3_basic.png,6135540f1af5ff3_basic,"\,^{*}d\,^{*}H=\kappa \,^{*}d\phi = J_B . \label{bfm19}",56,30,282,"\,^{*}d\,^{*}H=\kappa \,^{*}d\phi = J_B . \label{bfm19}",55,"[\, ,, ^, {, *, }, d, \, ,, ^, {, *, }, H, =, \kappa, , \, ,, ^, {, *, }, d, \phi, , =, , J, ...","[61, 13, 524, 553, 11, 555, 530, 61, 13, 524, 553, 11, 555, 41, 30, 264, 556, 61, 13, 524, 553, ...",42
8,92656e604bed774_basic.png,92656e604bed774_basic,{\phi''\over A} +{1\over A}\left( -{1\over 2}{A'\over A}+2{B'\over B}+{2\over r}\right)\phi'-{2 ...,159,80,817,{\phi''\over A} +{1\over A}\left( -{1\over 2}{A'\over A}+2{B'\over B}+{2\over r}\right)\phi'-{2 ...,159,"[{, \phi, ', ', \over, , A, }, , +, {, 1, \over, , A, }, \left, (, , -, {, 1, \over, , 2, }...","[553, 377, 8, 8, 365, 556, 34, 555, 556